## Lecture 10

## Python Parallel Computing - Part 01

### Apr 5, 2023


Part of this lecture is based on this material from previous years: https://nyu-cds.github.io/python-mpi/




You will need **mpi4py** 

To install: [https://mpi4py.readthedocs.io/en/stable/install.html](https://mpi4py.readthedocs.io/en/stable/install.html)


Run the following in terminal: 

1. sudo apt install libopenmpi-dev

2. pip install mpi4py


or 

1. brew install open-mpi

2. pip install mpi4py




### Two basic approaches

<img src="./figs/shared_memory.png" alt="shared_memory" style="width: 450px;"/>


<img src="./figs/distributed_memory.png" alt="distributed_memory" style="width: 500px;"/>

## Parallelization and Amdahl's law

* Want to leverage parallelization as much as possible
* Often we cannot obtain perfect (linear) speedups, e.g., communication or global logic
* Amdahl's law is a simple law to get an idea of the speedup:
    - $N$: number of processors
    - $P$: fraction of program that can be parallelized

$$
speedup = \frac{1}{(1 - P) + \frac{P}{N}}
$$

<img src="https://nyu-cds.github.io/python-mpi/fig/01-amdahls-law.png" alt="amdahls" style="width: 300px;"/>

---

### __MPI__ (Message Passing Interface) 

- Widely used standard


- For programming **distributed-memory**, **multiple instruction**--**multiple data** (MIMD) systems


#### __Point to point Communication__

Processes should coordinate their activities by explicitly sending and receiving messages

MPI operates as follows:
- Process A decides a message needs to be sent to process B.
- Process A packs up all of its necessary data into a buffer(**= a piece of memory**) for process B.
- Process A indicates that the data should be sent to process B by calling the _Send_ function.
- Process B needs to acknowledge it wants to receive the message by calling the _Recv_ function.

Every time a process sends a message, there must be a process that also indicates it wants to receive the message, therefore, calls to _Send_ and _Recv_ are always paired.



<img src="./figs/send_receive.png" alt="distributed_memory" style="width: 400px;"/>


### The number of processes 

- Is **fixed** when an MPI program is started 

- Each of the processes is assigned a unique integer starting from 0. 

- This integer is know as the **rank** of the process and is how each process is identified when sending and receiving messages (we will refer to rank K process as "process K").

- **MPI processes** are arranged in logical collections known as **communicators**. 

- There is one special communicator (**MPI.COMM_WORLD**) that exists when an MPI program starts, which contains all the processes in the MPI program. 


- MPI provides a few **methods** on a communicator:


> Get_size() - returns the total number of processes contained in the communicator (the size of the communicator).

> Get_rank() - returns the rank of the calling process within the communicator, between 0 and (size-1)

> Send() - sends content to a process

> Recv() - receives content from a process



In [2]:
from mpi4py import MPI

comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()
print('hello world: size = %d, rank = %d' % (size, rank))

hello world: size = 1, rank = 0


In [4]:
%%writefile mpi1.py
#####
# writing the code in the mpi1.py file
#####

from mpi4py import MPI

comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()
print('hello world: size = {}, rank = {}'.format(size, rank))

Writing mpi1.py


In [10]:
#####
# running MPI from the terminal with n=4 processes
# does not work in notebook for me for some reason, if so you can run in the terminal
#####

!mpiexec -n 4 python mpi1.py
# > -n : number of cummunicators, not cores
# > why > my cores work?

hello world: size = 4, rank = 1
hello world: size = 4, rank = 2
hello world: size = 4, rank = 3
hello world: size = 4, rank = 0


---
### One MPI program, multiple MPI processes
Making each process to perform a different computation 

In [11]:
%%writefile mpi2.py

from mpi4py import MPI
rank = MPI.COMM_WORLD.Get_rank()

a = 8.0
b = 4.0

print('Process rank',rank)
# ways to do different things in one parallel computing
if rank == 0:
        print("addition:", a + b)

if rank == 1:
        print("multiplication:", a * b)

if rank == 2:
        print("maximum:", max(a,b))
        
if rank == 3:
        print("doing nothing:")

Writing mpi2.py


In [12]:
!mpiexec -n 4 python mpi2.py

Process rank 3
doing nothing:
Process rank 0
addition: 12.0
Process rank 1
multiplication: 32.0
Process rank 2
maximum: 8.0


---
### Point-to-point communication
Message passing involves two processes: a **sender** and a **receiver** (commands _Send_ and _Recv_).

In [14]:
%%writefile mpi3.py
#####
# Sending a message from one process to another
#####
import numpy

from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()

randNum = numpy.zeros(1)

if rank == 1:
        print("part of Process", rank, "- before receiving has the number", randNum[0])
        # generates a numpy array with one element unif. distr. from [0,1)
        randNum = numpy.random.rand(1)
        print("part of Process", rank, "- drew the number", randNum[0])
        comm.Send(randNum, dest=0) # dest: rank number of the destination process
        
if rank == 0:
        print("part of Process", rank, "- before receiving has the number", randNum[0])
        comm.Recv(randNum, source=1)
        print("part of Process", rank, "- received the number", randNum[0])

Writing mpi3.py


In [15]:
!mpiexec -n 2 python mpi3.py

part of Process 1 - before receiving has the number 0.0
part of Process 1 - drew the number 0.1457764072198401
part of Process 0 - before receiving has the number 0.0
part of Process 0 - received the number 0.1457764072198401


In [ ]:
%%writefile mpi4.py
#####
# Sending a message to a process and receiving a message back
#####

import numpy
from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()

randNum = numpy.zeros(1) 

if rank == 1:
        randNum = numpy.random.rand(1)
        print("Process", rank, "drew the number", randNum[0])
        comm.Send(randNum, dest=0)
        comm.Recv(randNum, source=0)
        print("Process", rank, "received the number", randNum[0], "from process 0")
        
if rank == 0:
        print("Process", rank, "before receiving has the number", randNum[0])
        comm.Recv(randNum, source=1)
        print("Process", rank, "received the number", randNum[0], "from process 1")
        randNum *= 20
        comm.Send(randNum, dest=1) 

<img src="./figs/send_receive_mul2.png" style="width: 400px;"/>

In [ ]:
!mpiexec -n 2 python mpi4.py


The receiving process does not always need to specify the source when issuing a Recv.

Instead, the process can accept **any message** that is being sent by another process. This is done by setting the source to **MPI.ANY_SOURCE**.

In [ ]:
%%writefile mpi5.py
#####
# Sending a message to a process and receiving a message back from MPI.ANY_SOURCE
#####

import numpy
from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()

randNum = numpy.zeros(1) 

if rank == 1:
        randNum = numpy.random.random_sample(1)
        print("Process", rank, "drew the number", randNum[0])
        comm.Send(randNum, dest=0)
        comm.Recv(randNum, source=MPI.ANY_SOURCE)
        print("Process", rank, "received the number", randNum[0])
        
if rank == 0:
        print("Process", rank, "before receiving has the number", randNum[0])
        comm.Recv(randNum, source=MPI.ANY_SOURCE)
        print("Process", rank, "received the number", randNum[0])    
        randNum *= 2
        comm.Send(randNum, dest=1)

In [ ]:
!mpiexec -n 2 python mpi5.py

---

Sometimes there are cases when a process might have to **send many different types of messages to another process**. Instead of having to go through extra measures to differentiate all these messages, MPI allows senders and receivers to also **specify message IDs (known as tags)** with the message. The receiving process can then request a message with a certain tag number and messages with different tags will be buffered until the process requests them.

```python
Comm.Send(buf, dest=0, tag=0)
Comm.Recv(buf, source=0, tag=0, status=None)
```

The _status_ can provide useful information
```python
info = MPI.Status()
source = info.Get_source()
tag = info.Get_tag()
count = info.Get_elements()
size = info.Get_count()
```

In [ ]:
%%writefile mpi_tag.py
from mpi4py import MPI

comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank() 

data1 = None
data2 = None

if rank == 0:
    data1 = ('a','b', 'c', 'd')
    data2 = (1, 2, 3, 4)
    
    comm.send(data1, dest=1, tag=0)    # tag: the unique ID  
    comm.send(data2, dest=1, tag=1)    # -> so you know recv of tag=1 get info from send of tag=1
    
    

elif rank == 1:
    print('On Process',rank,'before recv: data1 = ', data1)
    print('On Process',rank,'before recv: data2 = ', data2)
    
    data1 = comm.recv(source=0, tag=0)   
    data2 = comm.recv(source=0, tag=1)
    
    print('On Process',rank,'after  recv: data1 = ', data1)
    print('On Process',rank,'after  recv: data2 = ', data2)
    

In [ ]:
!mpiexec -n 2 python mpi_tag.py

In [16]:
%%writefile mpi_status.py
#####
# Sending a message from one process to another
#####


import numpy
from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()

info = MPI.Status()
# print("info: ", info)
# > will print twice because every core run through everythin

randNum = numpy.zeros(1)

if rank == 1:
        randNum = numpy.random.random_sample(1)
        print("Process", rank, "drew the number", randNum[0])
        comm.Send(randNum, dest=0)

if rank == 0:
        print("Process", rank, "before receiving has the number", randNum[0])
        comm.Recv(randNum, source=1, status=info)
        print("Process", rank, "received the number", randNum[0], "from Process", info.Get_source())
        
        
        

Writing mpi_status.py


In [18]:
!mpiexec -n 2 python mpi_status.py

Process 0 before receiving has the number 0.0
Process 0 received the number 0.17917842612568957 from Process 1
Process 1 drew the number 0.17917842612568957



### Non-blocking Communication

In the previous examples, the sender and receiver are not able to perform any action when sending or receiving a message. This can waste computation time while waiting for the call to complete. 

__Non-blocking communcation__ avoids this issue by using the _Isend_ and _Irecv_ methods, which start to send and receive operations and _then return immediately to continue computation_.

The completion of a send or receive operation can be managed using the _Test_, _Wait_, and _Cancel_ methods.

- test: keep trying until get result


In [32]:
%%writefile mpi6.py
#####
# this code is similar to mpi3.py, 
# but it uses Wait to block the processes
#####

import numpy
from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()

randNum = numpy.zeros(1)

if rank == 1:
        randNum = numpy.random.random_sample(1)
        print("Process", rank, "drew the number", randNum[0])
        
        req = comm.Isend(randNum, dest=0)    # I = immediate, don't care about when to send
        # req.Wait()    # wait until sth is received by another node
        
        print('something here')
        
if rank == 0:
        print("Process", rank, "before receiving has the number", randNum[0])
        
        req = comm.Irecv(randNum, source=1)  # I = immediate, don't care about receive or not
        # req.Wait() # wait until receive sth
        while not req.Test():
            print('waiting')
        
        print("Process", rank, "received the number", randNum[0])

Overwriting mpi6.py


In [33]:
!mpiexec -n 2 python mpi6.py

Process 0 before receiving has the number 0.0
waiting
Process 0 received the number 0.33216272505379907
Process 1 drew the number 0.33216272505379907
something here


### Overlap communication


**Example:** Process 1 overlaps a computation with sending the message and receiving the reply. The computation divides randNum by 10 and prints the result.

In [42]:
%%writefile mpi7.py
#####
# overlap communication
#####

import numpy
from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()

randNum = numpy.zeros(1) 

if rank == 1:
#         randNum = numpy.random.random_sample(1)
        randNum = numpy.array([50], dtype=numpy.float64)
        print("Process", rank, "drew the number", randNum[0])
        
        comm.Isend(randNum, dest=0)
        
        randNum[0] /= 10 # overlap communication -> modify 
        print("Process", rank, "number in overlap communication =", randNum[0])
        
        req = comm.Irecv(randNum, source=0)  # the above randNum is replaced
        req.Wait()
        print("Process", rank, "received the number", randNum[0])

if rank == 0:
        print("Process", rank, "before receiving has the number", randNum[0])
        req = comm.Irecv(randNum, source=1)
        req.Wait()
        print("Process", rank, "received the number", randNum[0])
        randNum *= 2
        comm.Isend(randNum, dest=1)

Overwriting mpi7.py


In [43]:
!mpiexec -n 2 python mpi7.py

Process 0 before receiving has the number 0.0
Process 0 received the number 50.0
Process 1 drew the number 50.0
Process 1 number in overlap communication = 5.0
Process 1 received the number 100.0
